In [110]:
import os

import pandas as pd
import polars as pl

import numpy as np

import glob
import kaggle_evaluation.nfl_inference_server
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import polars as pl
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [111]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
for dirname, _, filenames in os.walk('/kaggle/input/nfl-combine-results-dataset-2000-2022'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/nfl-combine-results-dataset-2000-2022/2016_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2011_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2022_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2021_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2015_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2004_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2019_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2012_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2005_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2003_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2010_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2007_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2008_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2006_combine.csv
/kaggle/input/nfl-combine-results-

In [112]:
def processPolars(test_polars, combine_polars):
    test_polars = test_polars.with_columns(
        [pl.col("player_birth_date").str.to_datetime(format="%Y-%m-%d").alias("datetime"),
    ]
    ).drop("player_birth_date").with_columns(
        [pl.col("datetime").dt.year().alias("player_born_year")]
    ).drop("datetime").with_columns(
        encoded_play_direction=pl.col("play_direction").str.encode("hex")
    ).drop("play_direction").with_columns(
        play_direction=pl.col("encoded_play_direction").str.to_integer(base=16, strict=False)
    ).drop("encoded_play_direction").with_columns(
        test_polars["player_role"].to_dummies()
    ).drop("player_role").with_columns(
        test_polars["player_position"].to_dummies()
    ).drop("player_position").with_columns(
        test_polars["player_side"].to_dummies()
    ).drop("player_side").with_columns(
        pl.col("player_height").str.extract(r"(\d+)-", 1).cast(pl.Int32).alias("feet"),
        pl.col("player_height").str.extract(r"-(\d+)", 1).cast(pl.Int32).alias("inches"),
    ).with_columns(
        (pl.col("feet") * 12 + pl.col("inches") + 2).alias("height")
    ).drop("player_height").drop("inches").drop("feet")
    
    # Define expected columns from training
    EXPECTED_POSITION_COLS = [
        'player_position_CB', 'player_position_DE', 'player_position_DT',
        'player_position_FB', 'player_position_FS', 'player_position_ILB',
        'player_position_MLB', 'player_position_NT', 'player_position_OLB',
        'player_position_QB', 'player_position_RB', 'player_position_S',
        'player_position_SS', 'player_position_TE', 'player_position_WR'
    ]
    
    EXPECTED_ROLE_COLS = [
        'player_role_Defensive Coverage', 'player_role_Other Route Runner',
        'player_role_Passer', 'player_role_Targeted Receiver'
    ]
    
    EXPECTED_SIDE_COLS = ['player_side_Defense', 'player_side_Offense']
    
    # Remove any extra columns not in training
    for col in test_polars.columns:
        if col.startswith('player_position_') and col not in EXPECTED_POSITION_COLS:
            test_polars = test_polars.drop(col)
        if col.startswith('player_role_') and col not in EXPECTED_ROLE_COLS:
            test_polars = test_polars.drop(col)
        if col.startswith('player_side_') and col not in EXPECTED_SIDE_COLS:
            test_polars = test_polars.drop(col)
    
    # Add missing columns as zeros
    for col in EXPECTED_POSITION_COLS + EXPECTED_ROLE_COLS + EXPECTED_SIDE_COLS:
        if col not in test_polars.columns:
            test_polars = test_polars.with_columns(pl.lit(0).alias(col))
    
    # Now join with combine data
    combined_test_polars = test_polars.join(
        combine_polars, on="player_name", how="full"
    ).drop("player_name_right").with_columns(
        encoded_player_name=pl.col("player_name").str.encode("hex")
    ).with_columns(
        player_name=pl.col("encoded_player_name").str.to_integer(base=16, strict=False)
    ).drop("encoded_player_name").drop("Ht").drop("Pos").drop("player_weight").with_columns(
        (pl.col("y") / 53.3).alias("y"),
        (pl.col("x") / 120).alias("x"),
        (pl.col("ball_land_x") / 120).alias("ball_land_x"),
        (pl.col("ball_land_y") / 53.3).alias("ball_land_y"),
    ).drop("School")

    for i in range(4):
        combined_test_polars = combined_test_polars.with_columns(
        pl.lit(0.0).alias(f'dummy_feature_{i}')
    )
    
    return combined_test_polars

In [113]:
def processPolarsOutputs(test_polars, combine_polars):
    test_polars = test_polars.with_columns(
        #(pl.col("y") / 53.3).alias("y"),
        #(pl.col("x") / 120).alias("x")
    )
    return test_polars

In [114]:
def create_sequences(input_df, output_df):
    """
    Create encoder-decoder sequences from input and output dataframes
    
    Args:
        input_df: Tracking data BEFORE ball is thrown (encoder input)
        output_df: Tracking data WHILE ball is in air (decoder target)
    
    Returns:
        List of sequence dictionaries
    """
    # Define feature columns
    temporal_features = ['x', 'y', 's', 'a', 'dir', 'o']
    
    static_feature_cols = [
        'height', 'Wt', '40yd', 'Vertical', 'Bench', 
        'Broad Jump', '3Cone', 'Shuttle', 'player_born_year',
        'absolute_yardline_number', 'ball_land_x', 'ball_land_y',
        'play_direction'
    ] + [col for col in input_df.columns if col.startswith(
        ('player_position_', 'player_role_', 'player_side_'))]
    
    # Convert to lazy if not already
    input_lazy = input_df.lazy() if not isinstance(input_df, pl.LazyFrame) else input_df
    output_lazy = output_df.lazy() if not isinstance(output_df, pl.LazyFrame) else output_df
    
    # Sort both dataframes
    input_sorted = input_lazy.sort(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    output_sorted = output_lazy.sort(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    
    # Group columns
    group_cols = ['game_id', 'play_id', 'nfl_id']

    #works since data is now in temporal format
    
    # Pre-compute aggregations using Polars' efficient group_by
    input_agg = input_sorted.group_by(group_cols, maintain_order=True).agg([
        *[pl.col(feat).alias(f'temporal_{feat}') for feat in temporal_features],
        *[pl.col(col).first().alias(f'{col}_static') for col in static_feature_cols],
        pl.len().alias('num_input_frames')
    ])
    
    output_agg = output_sorted.group_by(group_cols, maintain_order=True).agg([
        pl.col('x').alias('decoder_x'),
        pl.col('y').alias('decoder_y'),
        pl.len().alias('num_output_frames')
    ])
    
    # Join the aggregated data and collect
    joined = input_agg.join(output_agg, on=group_cols, how='inner').collect()
    
    # Convert to sequences
    sequences = []
    for row in joined.iter_rows(named=True):
        # Extract temporal data - each feature is a list
        temporal_data = np.column_stack([
            row[f'temporal_{feat}'] for feat in temporal_features
        ])
        
        # Extract static features
        static_data = np.array([row[f'{col}_static'] for col in static_feature_cols], dtype=np.float32)
        
        # Repeat static features for each input frame
        num_input_frames = row['num_input_frames']
        static_repeated = np.tile(static_data, (num_input_frames, 1))
        
        # Combine temporal + static
        encoder_input = np.concatenate([temporal_data, static_repeated], axis=1)
        
        # Decoder target
        decoder_target = np.column_stack([
            row['decoder_x'],
            row['decoder_y']
        ])
        
        sequences.append({
            'encoder_input': encoder_input,
            'decoder_target': decoder_target,
            'static_features': static_data,
            'num_input_frames': num_input_frames,
            'num_output_frames': row['num_output_frames'],
            'game_id': row['game_id'],
            'play_id': row['play_id'],
            'nfl_id': row['nfl_id']
        })
    
    return sequences

In [115]:
class EncoderDecoderRNN(nn.Module):
    def __init__(self, encoder_input_dim, static_dim, hidden_dim, num_layers=2):
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # ENCODER: Process input sequence (before ball thrown)
        self.encoder = nn.LSTM(
            input_size=encoder_input_dim,  # temporal (6) + static features
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.2 if num_layers > 1 else 0
        )
        
        # DECODER: Generate output sequence (ball in air)
        # Input: static features + previous position (for teacher forcing)
        self.decoder = nn.LSTM(
            input_size=static_dim + 2,  # static features + (x, y) from previous step
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.2 if num_layers > 1 else 0
        )
        
        # Output layer
        self.fc = nn.Linear(hidden_dim, 2)  # Predict (x, y)
    
    def forward(self, encoder_input, static_features, num_output_frames, 
                decoder_target=None, teacher_forcing_ratio=0.5):
        """
        encoder_input: (batch, input_seq_len, encoder_input_dim)
        static_features: (batch, static_dim)
        num_output_frames: int or list of ints
        decoder_target: (batch, output_seq_len, 2) - for teacher forcing during training
        """
        batch_size = encoder_input.size(0)
        device = encoder_input.device
        
        # ENCODE
        encoder_output, (hidden, cell) = self.encoder(encoder_input)
        # hidden, cell: (num_layers, batch, hidden_dim)
        
        # DECODE
        # Start with last position from encoder input
        decoder_input = encoder_input[:, -1, :2]  # Last (x, y) position
        
        # Prepare to collect outputs
        if isinstance(num_output_frames, int):
            max_output_frames = num_output_frames
        elif torch.is_tensor(num_output_frames):
            max_output_frames = num_output_frames.item() if num_output_frames.dim() == 0 else int(max(num_output_frames))
        else:
            max_output_frames = int(max(num_output_frames))
        
        outputs = []
        
        for t in range(max_output_frames):
            # Concatenate previous position with static features
            decoder_step_input = torch.cat([
                decoder_input,  # (batch, 2)
                static_features  # (batch, static_dim)
            ], dim=1).unsqueeze(1)  # (batch, 1, static_dim + 2)
            
            # Decoder step
            decoder_output, (hidden, cell) = self.decoder(
                decoder_step_input, (hidden, cell)
            )
            
            # Predict next position
            prediction = self.fc(decoder_output.squeeze(1))  # (batch, 2)
            outputs.append(prediction)
            
            # Teacher forcing: use actual target or prediction
            if decoder_target is not None and t < decoder_target.size(1):
                if torch.rand(1).item() < teacher_forcing_ratio:
                    decoder_input = decoder_target[:, t, :]  # Use actual
                else:
                    decoder_input = prediction  # Use prediction
            else:
                decoder_input = prediction
        
        # Stack outputs
        predictions = torch.stack(outputs, dim=1)  # (batch, max_output_frames, 2)
        
        return predictions
class EncoderDecoderDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        seq = self.sequences[idx]
        return {
            'encoder_input': torch.FloatTensor(seq['encoder_input']),
            'decoder_target': torch.FloatTensor(seq['decoder_target']),
            'static_features': torch.FloatTensor(seq['static_features']),
            'num_input_frames': seq['num_input_frames'],
            'num_output_frames': seq['num_output_frames']
        }

def collate_fn_encoder_decoder(batch):
    """Handle variable length sequences"""
    # Find max lengths
    max_input_len = max(item['num_input_frames'] for item in batch)
    max_output_len = max(item['num_output_frames'] for item in batch)
    
    batch_size = len(batch)
    encoder_input_dim = batch[0]['encoder_input'].shape[1]
    static_dim = batch[0]['static_features'].shape[0]
    
    # Initialize padded tensors
    padded_encoder_input = torch.zeros(batch_size, max_input_len, encoder_input_dim)
    padded_decoder_target = torch.zeros(batch_size, max_output_len, 2)
    static_features = torch.zeros(batch_size, static_dim)
    
    input_lengths = []
    output_lengths = []
    
    # Fill in data
    for i, item in enumerate(batch):
        input_len = item['num_input_frames']
        output_len = item['num_output_frames']
        
        padded_encoder_input[i, :input_len, :] = item['encoder_input']
        padded_decoder_target[i, :output_len, :] = item['decoder_target']
        static_features[i] = item['static_features']
        
        input_lengths.append(input_len)
        output_lengths.append(output_len)
    
    return {
        'encoder_input': padded_encoder_input,
        'decoder_target': padded_decoder_target,
        'static_features': static_features,
        'input_lengths': torch.LongTensor(input_lengths),
        'output_lengths': torch.LongTensor(output_lengths)
    }

# Training loop adjustment
def train_epoch(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    
    for batch in tqdm(dataloader, desc="Training"):
        encoder_input = batch['encoder_input'].to(device)
        decoder_target = batch['decoder_target'].to(device)
        static_features = batch['static_features'].to(device)
        output_lengths = batch['output_lengths']
        
        optimizer.zero_grad()
        
        # Forward with teacher forcing
        predictions = model(
            encoder_input, 
            static_features,
            output_lengths,
            decoder_target=decoder_target,
            teacher_forcing_ratio=0.5
        )
        
        # Masked loss
        loss = masked_mse_loss(predictions, decoder_target, output_lengths)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

In [116]:
class EncoderDecoderRNN(nn.Module):
    def __init__(self, encoder_input_dim, static_dim, hidden_dim, 
                 max_output_len=50, num_layers=2):
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # ENCODER
        self.encoder = nn.LSTM(
            input_size=encoder_input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.2 if num_layers > 1 else 0
        )
        
        # ATTENTION
        self.attention = Attention(hidden_dim)
        
        # POSITIONAL ENCODING
        self.positional_encoding = LearnedPositionalEmbedding(
            max_len=max_output_len,
            embedding_dim=32
        )
        
        # DECODER
        decoder_input_size = 2 + static_dim + hidden_dim + 32
        
        self.decoder = nn.LSTM(
            input_size=decoder_input_size,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.2 if num_layers > 1 else 0
        )
        
        # Output layer
        self.fc = nn.Linear(hidden_dim, 2)
    
    def forward(self, encoder_input, static_features, num_output_frames,
                decoder_target=None, teacher_forcing_ratio=0.5):  # ← 4 spaces, not 8!
        batch_size = encoder_input.size(0)
        device = encoder_input.device
        
        # ENCODE
        encoder_outputs, (hidden, cell) = self.encoder(encoder_input)
        
        # Start with last position
        decoder_input = encoder_input[:, -1, :2]
        
        # Prepare max output frames
        if isinstance(num_output_frames, int):
            max_output_frames = num_output_frames
        elif torch.is_tensor(num_output_frames):
            max_output_frames = num_output_frames.item() if num_output_frames.dim() == 0 else int(max(num_output_frames))
        else:
            max_output_frames = int(max(num_output_frames))
        
        outputs = []
        attention_weights_list = []
        
        for t in range(max_output_frames):
            # Get attention context using top decoder hidden state
            decoder_hidden_top = hidden[-1]  # (batch, hidden_dim)
            context, attn_weights = self.attention(encoder_outputs, decoder_hidden_top)
            
            # Get positional encoding for current timestep
            timestep_tensor = torch.full((batch_size,), t, dtype=torch.long, device=device)
            pos_encoding = self.positional_encoding(timestep_tensor)  # (batch, 32)
            
            # Concatenate all decoder inputs
            decoder_step_input = torch.cat([
                decoder_input,      # (batch, 2)
                static_features,    # (batch, static_dim)
                context,           # (batch, hidden_dim)
                pos_encoding       # (batch, 32)
            ], dim=1).unsqueeze(1)  # (batch, 1, decoder_input_size)
            
            # Decoder step
            decoder_output, (hidden, cell) = self.decoder(
                decoder_step_input, (hidden, cell)
            )
            
            # Predict next position
            prediction = self.fc(decoder_output.squeeze(1))  # (batch, 2)
            outputs.append(prediction)
            attention_weights_list.append(attn_weights)
            
            # Teacher forcing
            if decoder_target is not None and t < decoder_target.size(1):
                if torch.rand(1).item() < teacher_forcing_ratio:
                    decoder_input = decoder_target[:, t, :]
                else:
                    decoder_input = prediction
            else:
                decoder_input = prediction
        
        # Stack outputs
        predictions = torch.stack(outputs, dim=1)  # (batch, max_output_frames, 2)
        attention_weights = torch.stack(attention_weights_list, dim=1)
        
        return predictions, attention_weights

In [117]:
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        
        # Learnable weights
        self.W_encoder = nn.Linear(hidden_dim, hidden_dim)
        self.W_decoder = nn.Linear(hidden_dim, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        
    def forward(self, encoder_outputs, decoder_hidden):
        """
        encoder_outputs: (batch, seq_len, hidden_dim)
        decoder_hidden: (batch, hidden_dim)
        
        Returns:
        context: (batch, hidden_dim)
        attention_weights: (batch, seq_len)
        """
        seq_len = encoder_outputs.size(1)
        
        # Expand decoder hidden to match encoder seq_len
        decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, seq_len, 1)
        # (batch, seq_len, hidden_dim)
        
        # Calculate attention scores
        energy = torch.tanh(
            self.W_encoder(encoder_outputs) + self.W_decoder(decoder_hidden)
        )  # (batch, seq_len, hidden_dim)
        
        attention_scores = self.V(energy).squeeze(-1)  # (batch, seq_len)
        
        # Softmax to get attention weights
        attention_weights = torch.softmax(attention_scores, dim=1)
        # (batch, seq_len)
        
        # Weighted sum of encoder outputs
        context = torch.bmm(
            attention_weights.unsqueeze(1),  # (batch, 1, seq_len)
            encoder_outputs  # (batch, seq_len, hidden_dim)
        ).squeeze(1)  # (batch, hidden_dim)
        
        return context, attention_weights

In [118]:
class LearnedPositionalEmbedding(nn.Module):
    def __init__(self, max_len, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(max_len, embedding_dim)
        
    def forward(self, timestep):
        """
        timestep: int or (batch,) tensor of timesteps
        """
        if isinstance(timestep, int):
            timestep = torch.tensor([timestep], device=self.embedding.weight.device)
        return self.embedding(timestep)

In [119]:
model = torch.load('/kaggle/input/modelwithattention/best_nfl_model (3).pth',weights_only="True")

In [120]:
static_dih = model['static_dim']
static_dih

34

In [121]:
awko_dim = model['hidden_dim']
awko_dim

32

In [122]:
encoder_input_dim = model['encoder_input_dim']
encoder_input_dim

40

In [123]:
train_path = '/kaggle/input/nfl-combine-results-dataset-2000-2022/'
combine_files = sorted(glob.glob(train_path + "*_combine.csv"))
combine_data = pd.concat([pd.read_csv(f) for f in combine_files], ignore_index=True)
#loads the combine data
combine_polars = pl.from_pandas(combine_data)
combine_polars = combine_polars.rename({'Player': 'player_name'})

In [ ]:
test_input = pl.read_csv('/kaggle/input/nfl-big-data-bowl-2026-prediction/test_input.csv')


In [126]:
    if torch.cuda.is_available():
        device = torch.device('cuda') # or 'cuda:0', 'cuda:1', etc.
    else:
        device = torch.device('cpu')
    
    new_model_input = processPolars(test_input,combine_polars)
    new_model_input.describe()
    new_model_input = new_model_input.fill_null(0)
    test_filled = test.fill_null(0)
    sequences = create_sequences(new_model_input,test_filled)

    test_dataset = EncoderDecoderDataset(sequences)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,
                            collate_fn=collate_fn_encoder_decoder, num_workers=2)
    max_frames_in_data = 0
    for batch in train_loader:
        max_frames_in_data = max(max_frames_in_data, max(batch['output_lengths']))
    

    checkpoint = torch.load('/kaggle/input/modelwithattention/best_nfl_model (3).pth',weights_only="True")
    encoder_input_dim = checkpoint['encoder_input_dim']
    static_dim = checkpoint['static_dim']
    hidden_dim = checkpoint['hidden_dim']
    num_layers = checkpoint['num_layers']

    model = EncoderDecoderRNN(
        encoder_input_dim=encoder_input_dim,
        static_dim=static_dim,
        hidden_dim=128,
        max_output_len=max_frames_in_data + 10,  # Add buffer
        num_layers=2
    ).to(device)

    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    all_predictions = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Predicting"):
            encoder_input = batch['encoder_input'].to(device)
            static_features = batch['static_features'].to(device)
            output_lengths = batch['output_lengths']
            
            predictions = model(encoder_input, static_features, output_lengths,
                              decoder_target=None, teacher_forcing_ratio=0.0)
            
            all_predictions.append(predictions.cpu())
    
    all_predictions = torch.cat(all_predictions, dim=0).numpy()
    pred_x = all_predictions[:, :, 0] * 120    # Denormalize x
    pred_y = all_predictions[:, :, 1] * 53.3   # Denormalize y
    
    # Flatten predictions to match test DataFrame format
    pred_x_flat = pred_x.flatten()[:len(test)]
    pred_y_flat = pred_y.flatten()[:len(test)]
    
    # CRITICAL FIX: Clone the test DataFrame and update x,y columns  
    # This preserves all original columns (game_id, play_id, nfl_id, frame_id, etc.)
    predictions = test.clone()
    predictions = predictions.with_columns([
        pl.Series('x', pred_x_flat.tolist()),
        pl.Series('y', pred_y_flat.tolist())
    ])

NameError: name 'test_input' is not defined

In [ ]:
def predict(test: pl.DataFrame, test_input: pl.DataFrame) -> pl.DataFrame | pd.DataFrame:
    """Replace this function with your inference code.
    You can return either a Pandas or Polars dataframe, though Polars is recommended for performance.
    Each batch of predictions (except the very first) must be returned within 5 minutes of the batch features being provided.
    """

    if torch.cuda.is_available():
        device = torch.device('cuda') # or 'cuda:0', 'cuda:1', etc.
    else:
        device = torch.device('cpu')
    
    new_model_input = processPolars(test_input,combine_polars)
    new_model_input.describe()
    new_model_input = new_model_input.fill_null(0)
    test_filled = test.fill_null(0)
    sequences = create_sequences(new_model_input,test_filled)

    test_dataset = EncoderDecoderDataset(sequences)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,
                            collate_fn=collate_fn_encoder_decoder, num_workers=2)
    max_frames_in_data = 0
    for batch in train_loader:
        max_frames_in_data = max(max_frames_in_data, max(batch['output_lengths']))
    

    checkpoint = torch.load('/kaggle/input/modelwithattention/best_nfl_model (3).pth',weights_only="True")
    encoder_input_dim = checkpoint['encoder_input_dim']
    static_dim = checkpoint['static_dim']
    hidden_dim = checkpoint['hidden_dim']
    num_layers = checkpoint['num_layers']

    model = EncoderDecoderRNN(
        encoder_input_dim=encoder_input_dim,
        static_dim=static_dim,
        hidden_dim=128,
        max_output_len=max_frames_in_data + 10,  # Add buffer
        num_layers=2
    ).to(device)

    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    all_predictions = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Predicting"):
            encoder_input = batch['encoder_input'].to(device)
            static_features = batch['static_features'].to(device)
            output_lengths = batch['output_lengths']
            
            predictions = model(encoder_input, static_features, output_lengths,
                              decoder_target=None, teacher_forcing_ratio=0.0)
            
            all_predictions.append(predictions.cpu())
    
    all_predictions = torch.cat(all_predictions, dim=0).numpy()
    pred_x = all_predictions[:, :, 0] * 120    # Denormalize x
    pred_y = all_predictions[:, :, 1] * 53.3   # Denormalize y
    
    # Flatten predictions to match test DataFrame format
    pred_x_flat = pred_x.flatten()[:len(test)]
    pred_y_flat = pred_y.flatten()[:len(test)]
    
    # CRITICAL FIX: Clone the test DataFrame and update x,y columns  
    # This preserves all original columns (game_id, play_id, nfl_id, frame_id, etc.)
    predictions = test.clone()
    predictions = predictions.with_columns([
        pl.Series('x', pred_x_flat.tolist()),
        pl.Series('y', pred_y_flat.tolist())
    ])
    
    assert isinstance(predictions, (pd.DataFrame, pl.DataFrame))
    assert len(predictions) == len(test)
    return predictions

In [ ]:
inference_server = kaggle_evaluation.nfl_inference_server.NFLInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/nfl-big-data-bowl-2026-prediction/',))